# DNS

NOTE - DNS Attacks with bind server doesn't work in Apple silicon (M1, M2) based Macs. So, use GitHub Codespaces or VMs with Intel/AMD processors.

- SEED Security Labs - [https://seedsecuritylabs.org/Labs_20.04/Networking/DNS/DNS_Local/](https://seedsecuritylabs.org/Labs_20.04/Networking/DNS/DNS_Local/)
- Domain Name Service (DNS) is a system that translates domain names to IP addresses
- DNS is a hierarchical and decentralized naming system for computers, services, or other resources connected to the Internet or a private network
- DNS is a key component of the Internet
- DNS is a client-server network communication system
- DNS is a distributed database that contains records for each domain name
- Due its importance, DNS is a common target for cyber attacks
    - DNS Spoofing, DNS Hijacking, DNS Amplification, DNS Flooding (DoS), DNS cache poisoning, etc.
- E.g. `www.google.com` is translated to `142.250.191.46` according to [https://www.nslookup.io/domains/google.com/webservers/](https://www.nslookup.io/domains/google.com/webservers/)

## DNS Hierarchical Structure, Zones and Servers

- Humans can remember domain names, but computers need IP addresses
- DNS is a hierarchical system that starts from the root domain and goes down to the top-level domain (TLD), second-level domain, subdomain, etc.
- Each domain is a zone and has a DNS server that is responsible for that zone
- DNS servers are classified as:
    - Root Servers: Servers that are responsible for the root zone (.)
    - TLD Servers: Servers that are responsible for the top-level domain zone
        - E.g. `.com`, `.org`, `.net`, `.gov`, `.edu`, `.uk`, `.fr`, `.de`, `.jp`, `.cn`, `.ru`, etc.
    - Second-level Servers: Servers that are responsible for a specific domain zone
        - E.g. `google.com`, `facebook.com`, `twitter.com`, `amazon.com`, etc.
- the official list of all the TLDs is maintained by the Internet Assigned Numbers Authority (IANA)
- The root servers are managed by the Internet Corporation for Assigned Names and Numbers (ICANN)
- The root servers are distributed around the world and are managed by different organizations
- The root servers are the entry point to the DNS system
- TLDs are managed by "designated organizations" or "sponsored organizations"
    - E.g. `.com` is managed by Verisign, `.org` is managed by Public Interest Registry, etc.
    - .gov is managed by the General Services Administration (GSA), .edu is managed by Educause, etc.
- Second-level domains are managed by the organization that owns the domain
- when a domain name is purchased, the TLD's designated managers add the information to the corresponding registry databases
    - GoDaddy, Namecheap, Google Domains, Tcows, eNom, etc.

## DNS Zones

- A DNS zone is a portion of the DNS namespace that is managed by a specific organization or administrator
- A zone may have manage one or more subdomains for a specific domain
- a multinational company may have multiple zones for each country
    - e.g., `us.company.com`, `uk.company.com`, `fr.company.com`, `de.company.com`, etc.
- each zone has a zone file that stores DNS records.

### Types of DNS Zones

#### Primary Zone

- holds the original read-write copy of the zone file.
- changes to DNS records (like adding new subdomains or IPs) are made here.
- Example: The authoritative DNS server for example.com holds the primary zone file.

#### Secondary Zone

- read-only copy of the primary zone.
- updated from the primary zone via zone transfers (AXFR or IXFR).
- used for **redundancy** and **load balancing**.

#### Stub Zone

- contains only essential records (like NS and SOA) to identify the authoritative servers for another zone.
- used for delegation and resolving dependencies between zones.

#### Forward Zone

- maps domain names → IP addresses.
- Example: example.com → 93.184.216.34

#### Reverse Zone

- maps IP addresses → domain names.
- Example: 93.184.216.34 → example.com

### A simple example.com zone file

```txt
$TTL 86400
@   IN  SOA  ns1.example.com. admin.example.com. (
        2025101301 ; Serial date based
        3600       ; Refresh; frequency of zone file updates from primary to secondary
        1800       ; Retry interval; wait before retrying a failed zone transfer
        1209600    ; Expire; time before secondary stops answering if it can't reach primary
        86400 )    ; Minimum TTL; default cache time for negative responses, e.g., name not found

    IN  NS   ns1.example.com. ; Authoritative nameserver
    IN  NS   ns2.example.com. ; Secondary nameserver

@   IN  A    93.184.216.34 ; A record for example.com
www IN  A    93.184.216.34 ; A record for www subdomain
mail IN  A    93.184.216.35 ; A record for mail subdomain
@   IN  MX   10 mail.example.com. ; mail exchange for example.com; priority 10
```

- `$TTL` (Time To Live) specifies how long the record is cached by DNS resolvers 24 hours here
- `;` indicates comments
- `@` represents the current zone (example.com)
- `IN` indicates the Internet class
- `SOA` (Start of Authority) record contains administrative information about the zone
- `NS` (Name Server) records specify the authoritative DNS servers for the zone
- `A` records map domain names to IPv4 addresses
- `MX` (Mail Exchange) records specify the mail servers for the domain


## DNS Query Process
- DNS query process is a recursive process:

1. When a client wants to resolve a domain name (www.company.net), it sends a query to the local DNS server
2. The local DNS server checks its cache for the domain name
3. If the domain name is not in the cache, the local DNS server sends a query to the root server
4. The root server doesn't know the IP address, it responds with the IP address of the TLD server
5. The local DNS server sends a query to the TLD server (.net)
6. If the TLD server doesn't know the IP address, it responds with the IP address of the second-level server (company.net)
7. The local DNS server sends a query to the second-level server (company.net)
8. The second-level server responds with the IP address of the domain name (www.company.net)

## Local DNS Server and File

- in Linux, the local DNS resolver depends on two files: `/etc/resolv.conf` and `/etc/hosts`

### /etc/resolv.conf
- The `/etc/resolv.conf` file contains the IP address of the local DNS server
- The local DNS server is usually the router or the ISP's DNS server
- if a host uses Dynamic Host Configuration Protocol (DHCP), the DHCP server automatically updates the `/etc/resolv.conf` file
- the `/etc/resolv.conf` file may contain multiple DNS servers

In [3]:
! cat /etc/resolv.conf

# Dynamic resolv.conf(5) file for glibc resolver(3) generated by resolvconf(8)
#     DO NOT EDIT THIS FILE BY HAND -- YOUR CHANGES WILL BE OVERWRITTEN
nameserver 192.168.35.2
search localdomain


### /etc/hosts

- `/etc/hosts` file contains the IP address and domain names
- file is used to resolve domain names before sending a query to the DNS server
- its text file with the format: `IP_address    domain_name`
- e.g.,

In [2]:
! cat /etc/hosts

##
# Host Database
#
# localhost is used to configure the loopback interface
# when the system is booting.  Do not change this entry.
##
127.0.0.1        localhost
255.255.255.255  broadcasthost
::1              localhost

## Dig Command

- `dig` (domain information groper) is a command-line tool for querying DNS servers
- `dig` is part of the `bind-utils` package
- `dig` is used to query DNS servers for information about domain names and IP addresses
- `dig` has an option to query a specific DNS server
- `dig` has an option to query a specific record type (A, AAAA, CNAME, MX, NS, SOA, TXT, etc.)

### Syntax

```bash
dig domain_name
dig domain_name NS
dig @ns4.google.com google.com
```

In [8]:
! dig @a.root-servers.net www.example.net


; <<>> DiG 9.20.8-6-Debian <<>> @a.root-servers.net www.example.net
; (2 servers found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 28576
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 13, ADDITIONAL: 27
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;www.example.net.		IN	A

;; AUTHORITY SECTION:
net.			172800	IN	NS	m.gtld-servers.net.
net.			172800	IN	NS	k.gtld-servers.net.
net.			172800	IN	NS	b.gtld-servers.net.
net.			172800	IN	NS	f.gtld-servers.net.
net.			172800	IN	NS	d.gtld-servers.net.
net.			172800	IN	NS	h.gtld-servers.net.
net.			172800	IN	NS	j.gtld-servers.net.
net.			172800	IN	NS	l.gtld-servers.net.
net.			172800	IN	NS	a.gtld-servers.net.
net.			172800	IN	NS	e.gtld-servers.net.
net.			172800	IN	NS	c.gtld-servers.net.
net.			172800	IN	NS	g.gtld-servers.net.
net.			172800	IN	NS	i.gtld-servers.net.

;; ADDITIONAL SECTION:
m.gtld-servers.net.	172

In [5]:
! dig @m.gtld-servers.net www.example.net


; <<>> DiG 9.20.8-6-Debian <<>> @m.gtld-servers.net www.example.net
; (2 servers found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 63797
;; flags: qr rd; QUERY: 1, ANSWER: 0, AUTHORITY: 2, ADDITIONAL: 5
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;www.example.net.		IN	A

;; AUTHORITY SECTION:
example.net.		172800	IN	NS	a.iana-servers.net.
example.net.		172800	IN	NS	b.iana-servers.net.

;; ADDITIONAL SECTION:
a.iana-servers.net.	172800	IN	A	199.43.135.53
a.iana-servers.net.	172800	IN	AAAA	2001:500:8f::53
b.iana-servers.net.	172800	IN	A	199.43.133.53
b.iana-servers.net.	172800	IN	AAAA	2001:500:8d::53

;; Query time: 39 msec
;; SERVER: 192.55.83.30#53(m.gtld-servers.net) (UDP)
;; WHEN: Tue Oct 14 20:04:59 MDT 2025
;; MSG SIZE  rcvd: 177



In [10]:
! dig @a.iana-servers.net www.example.net


; <<>> DiG 9.20.8-6-Debian <<>> @a.iana-servers.net www.example.net
; (2 servers found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 21358
;; flags: qr aa rd; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;www.example.net.		IN	A

;; ANSWER SECTION:
www.example.net.	300	IN	CNAME	www.example.net-v2.edgesuite.net.

;; Query time: 59 msec
;; SERVER: 199.43.135.53#53(a.iana-servers.net) (UDP)
;; WHEN: Tue Oct 14 20:09:43 MDT 2025
;; MSG SIZE  rcvd: 87



In [11]:
! dig @a.iana-servers.net example.net


; <<>> DiG 9.20.8-6-Debian <<>> @a.iana-servers.net example.net
; (2 servers found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 55746
;; flags: qr aa rd; QUERY: 1, ANSWER: 4, AUTHORITY: 0, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;example.net.			IN	A

;; ANSWER SECTION:
example.net.		30	IN	A	23.215.0.135
example.net.		30	IN	A	23.215.0.141
example.net.		30	IN	A	23.220.75.234
example.net.		30	IN	A	23.220.75.235

;; Query time: 63 msec
;; SERVER: 199.43.135.53#53(a.iana-servers.net) (UDP)
;; WHEN: Tue Oct 14 20:10:03 MDT 2025
;; MSG SIZE  rcvd: 104



## DNS Attack Setup

![DNS Attack Setup with Containers](./resources/NetworkSetup-DNS.png)
- see `course/demos/DNS_Attack` folder for the setup

### Configure User Machine
- configure user machine with ip 10.9.0.5 to use 10.9.0.53 as DNS server
- change the `/etc/resolv.conf` file to use the DNS server

```bash
echo "nameserver 10.9.0.53" > /etc/resolv.conf
```

### Configure DNS Server

- dns-server-53 container at 10.9.0.53 is the DNS server for our experimental setup
- use widely used DNS server software BIND (Berkley Internet Name Domain) by UC Berkeley
- install `bind9` package released in 2000
- `bind9` is already installed on the `dns-server-53` and `attacker-dns-153` containers


```bash
apt list --installed | grep bind9
sudo apt install bind9
```

- configure bind9 by updating `/etc/bind/named.conf.options` file
- dump file - tells where the cache is dumped to
- can use rndc (Remote Name Daemon Control) to mange `bind9`
- to dump cache to a file and empty it from server's cache, use the following rndc commands

```bash
rndc dumpdb -cache
rndc flush
```

- disable DNSSEC and turn off DNSSEC validation
- use fixed static query source port; random query source port is default for security
    - random source port makes attacks harder to forge DNS response
- start the bind9 service or restart the containers

```bash
service named start
```

### Configure Attacker's Nameserver

- use `attacker32.com` as the attacker's domain (owned by SEED Lab) but not hosting any site
- set up the authoritative nameserver for this domain on the attacker's nameserver container

#### Create zones

- setup forward look up zone; uses RFC 1035 format
- configure `named.conf` with zone configurations
- see the `named.conf` file in the `attackers_ns` directory
    - `@` stands for the zone name - `attacker32.com`
- the `file` keyword in the zone definition is called zone file
- the `named.conf` file is copied into the attacker's nameserver container

#### Setup the forward and reverse lookup zone files

- the zone file is where the actual DNS resolution is stored
- in `/etc/bind/` the following files are copied into attacker's nameserver container
    - forward lookup zone file: `zone_attacker32.com`
    - forward lookup zone file: `zone_example.com`
    - reverse lookup zone file: `zone_192.168.0`
- see the contents of the files and comments after `;` for explanation of each line

## Test the setup

```bash
docker-compose up -d
docker ps
docker exec -it user-5 bash
```
- make any changes if necessary and restart the BIND server or restart the container
- from the `user-5` container, run the `dig` command to ask the nameserver `10.9.0.153`
- Note that running dig to any other sites in CMU network does NOT work!

```bash
dig @10.9.0.153 www.attacker32.com
dig @10.9.0.153 -x 192.168.0.101
```

```bash
dig @10.9.0.153 www.attacker32.com
dig @10.9.0.153 -x 192.168.0.101
```

#### Add Forward Zones to Local DNS Server

- while testing above, we've to specify the specific `@NSIP` when running the dig
- if we don't, the local DNS server will be used
- since `www.attacker32.com` is not registered with `.com` server; the local DNS gets the official nameserver for the `attacker32.com` and NOT `10.9.0.153`
- two problems: private IP as DNS server and only the owner of the domain (author of seedlabs) can register `attacker32.com` with `.com` root server
- to solve it, we add a forward zone in the local DNS server (`10.9.0.53`)
- so any queries to `attacker32.com` will be forwarded to `10.9.0.153`
- also need to add `empty-zone-enable on` so the empty zone is created for private/reserved IP prefix

#### Testing without @NSIP option

- we should see the same result as with using @NSIP

```bash
dig www.attacker32.com
dig -x 192.168.0.101
```

### Constructing DNS Packets with Scapy

- we need to construct DNS packets (mostly spoofed replies)
- DNS uses IP/UDP protocols
- DNS has several sections: question, answer, authority and additional
- DNS header contains a transaction ID which must be the same in the response packet as in the query packet; or the query will be discarded
- also has 16-bit flags: 0-15
- `qr`, `aa`, `rd` flags are important for security experiments
- `qr`: 1-bit field; value 0-1; 0 means a query and 1 means a response packet
- `aa`: specifies whether the answer is authoritative or not (1 means authoritative)
- `rd`: specifies whether recursion is desired (only for queries)
- let's see the DNS packet fields using Scapy

In [1]:
! python -c "from scapy.all import *; ls(DNS)"

length     : ShortField (Cond)                   = ('None')
id         : ShortField                          = ('0')
qr         : BitField  (1 bit)                   = ('0')
opcode     : BitEnumField                        = ('0')
aa         : BitField  (1 bit)                   = ('0')
tc         : BitField  (1 bit)                   = ('0')
rd         : BitField  (1 bit)                   = ('1')
ra         : BitField  (1 bit)                   = ('0')
z          : BitField  (1 bit)                   = ('0')
ad         : BitField  (1 bit)                   = ('0')
cd         : BitField  (1 bit)                   = ('0')
rcode      : BitEnumField                        = ('0')
qdcount    : DNSRRCountField                     = ('None')
ancount    : DNSRRCountField                     = ('None')
nscount    : DNSRRCountField                     = ('None')
arcount    : DNSRRCountField                     = ('None')
qd         : DNSQRField                          = ('<DNSQR  |>')
an     

### DNS Records

- Scapy defines two classes: DNSRR and DNSQR
- DNSRR - for the answer and authority records
- DNSQR - for the question record

#### Question Record

| Name | Record Type | Class |
| :--- | :--- | :--- |
|www.example.com | "A" Record 0x0001 | Internet 0x0001|

- let's see the question record using Scapy

In [2]:
! python -c 'from scapy.all import *; ls(DNSQR)'

qname      : DNSStrField                         = ("b'www.example.com.'")
qtype      : ShortEnumField                      = ('1')
qclass     : ShortEnumField                      = ('1')


#### Answer Record

| Name | Record Type | Class | Time To Live | Data Length | Data: IP Add |
| :--- | :--- | :--- | :--- | :--- | :---|
| www.example.com| "A" | Internet | 0x200 (s) | 0x0004 | 1.2.3.4 |


#### Authority Record

| Name | Record Type | Class | Time To Live | Data Length | Data: IP Add |
| :--- | :--- | :--- | :--- | :--- | :---|
| example.com| "NS" | Internet | 0x200 (s) | 0x0013 | ns.example.com |

- let's look in the DNSRR record details using Scapy


In [5]:
! python -c 'from scapy.all import *; ls(DNSRR)'

rrname     : DNSStrField                         = ("b'.'")
type       : ShortEnumField                      = ('1')
rclass     : ShortEnumField                      = ('1')
ttl        : IntField                            = ('0')
rdlen      : FieldLenField                       = ('None')
rdata      : MultipleTypeField (IPField, IP6Field, DNSStrField, DNSTextField, StrLenField) = ("b''")


### Sending a DNS Query using Scapy

- use DNS server provided by Google: 8.8.8.8
- DNS listens on UDP port 53
- see the script in `course/demos/DNS_Attack/volumes/send_dns_query.py`
- run the script using `sudo`
- NOTE: for some reason the script doesn't get the response when running from a container!
- run from the host machine with scapy installed!

In [10]:
! cat ../demos/DNS_Attack/volumes/send_dns_query.py

#! /usr/bin/env python3

from scapy.all import *

DNS_SERVER_IP = "8.8.8.8"
QUERY_DOMAIN = "www.coloradomesa.edu"


def send_dns_query():
    # Define the target IP address
    IPpkt = IP(dst=DNS_SERVER_IP)
    UDPpkt = UDP(dport=53)
    # Define the DNS query
    DNSpkt = DNS(id=100, qr=0, qdcount=1, qd=DNSQR(qname=QUERY_DOMAIN))
    # Combine the IP, UDP and DNS packets
    request = IPpkt/UDPpkt/DNSpkt
    # Send the request and receive the response
    print("Sending: ", request.show())
    response = sr1(request)
    # Print the response
    print(response.show())


if __name__ == "__main__":
    send_dns_query()


## DNS Attack Surfaces


### Attacks on compromised machines

- if the attackers gain root privilege on a system, they can do a lot of damages
- they can simply modify the two local configuration files that DNS depends on:
    - `/etc/resolv.conf` and `etc/hosts`
- by modifiying /etc/resolv.conf, attackers can use a malicious DNS server as the machine's local DNS server
- by modifying /etc/hosts, attackers can add new records to the file, providing the IP addresses for some selected domains

### Attacks on user machine

- when a user machine sends out a DNS query to its local DNS server, attacker can sniff and spoof reply

### Local DNS cache poisoning attack

- when the local DNS server sends out iterative queries to get an answer from the authoritative servers on the Internet, attackers can send out spoofed replies to the local DNS server
- the spoofed reply is cached by local DNS server essentially poisoning the local cache with lasting and wider damages to the local network

### Attacks from malicious DNS server

- malicious DNS server run by attackers can send additional information in the answer section of the response
- if there's no restriction on what information can be placed in authority and additional sections of answer packets, attackers can use them to their advantage e.g., to provide fraudulent information
- this kind of attacks were effective in earlier versions of DNS server


## Local DNS Poisoning Attack

![DNS Cache Poisoning Attack](./resources/DNS-Poisoning-Attack.png)


### Flush the cache if necessay

- check the cache and flush it if necessary
- the cache on the DNS server may already have the information for the domain `www.example.com`
- flush and clear the local DNS server (dns-server-53) cache to demonstrate the attack
- in real attacks, one may have to wait for the cache to expire or use some technique to bypass it

```bash
rndc dumpdb -cache
cat /var/cache/bind/dump.db
rndc flush # deelete the cache
rndc dumpdb -cache
cat /var/cache/bind/dump.db
```

### Potential Issues

- when conducting the attack using containers, sometimes the sniffing and spoofing doesn't work, likely because the of delay due to virtual networks used in Docker
- VMs do not have the issues
- if it doesn't work keep trying multiple times, each time flushing the local DNS server's cache
- can also intentionally slow down the traffic going to the outside using **traffic control**, `tc`,  command on the router, so the authentic reply doesn't come back in time
- note that router container has two interfaces, `eth0` and `eth1`

```bash
ip -br address
```

- make sure to run the `tc` on the the interface that's connected to external network `10.8.0.0/24`

```bash
# delay the network traffic by 100 ms
tc qdisc add dev <interface> root netem delay 100ms

# delete the tc entry
tc qdisc del dev <interface> root netem

# show all the tc entries
tc qdisc show dev <interface>

```

### Attack a Specific User with MitM

#### NOTE: Don't use CMU WiFi; Use personal hostspot when on campus to run the following attacks!
#### Don't visit www.example.net from firefox before DNS cache poisoning attack
- if you do, restart the containers to flush the cache from firefox; so it tries to take you to Google.com's 404 Error page!

- attackers can sniff the unencrypted network traffic and simply send a forged reeply to the user machine
- attackers need to sniff the query packet and note the following information:
    1. UDP source port number
    2. the transaction ID of the query
    3. the question in the query
- let's sniff and spoof DNS request from a user
- configure and run `DNS_Attack/volumes/dns_sniff_spoof.py` on the attacker container
- on the user's machine:
    - `dig www.example.net`
    - `ping www.example.net`
- it should resolve to the spoofed IP address in the script


### Local DNS Server Cache Poisoning Attack
- when a local DNS server receives DNS query, it first checks in its cache
- if the query is not found in local cache, it will further send out queries to other DNS servers
- attackers can sniff the unencrypted network traffic and simply send a forged reeply to the local DNS server
- attackers can cause more damage by poisioning the cache of the local DNS server
- attackers need to sniff the query packet and note the following information:
    1. UDP source port number
    2. the transaction ID of the query
    3. the question in the query
    
- let's poison the cache of local DNS server (10.9.0.53) in the `dns-server-53` container setup
- configure and run `DNS_Attack/volumes/poison_dns_cache.py` on the attacker container
- make sure to find and change the interface name in the script using the following command

```bash
ip -br address
```

- flush the cache and clear/delete /var/cache/bind/dump.db

```bash
rndc flush
rndc dumpdb -cache
grep -B 1 example /var/cache/bind/dump.db
```

- on the user machine run `dig` or `ping`

```bash
dig www.example.net
ping www.example.net
```

- if you see `208.67.220.220` in answer, the user got the forged reply!
- run `rndc dumpdb -cache` command to see if the cache is poisoned on the DNS server
- the cache set to be dumped to `/var/cache/bind/dump.db` in the `named.conf.options` file

```bash
rndc dumpdb -cache
grep -B 1 example /var/cache/bind/dump.db
```

#### All Systems in LAN are Affected

- all the systems configured to use `dns-server-53` are affected by the poisoned cache
- open `firefox` container and visit `www.example.net`
- From host browser, go to `localhost:5800`
- You'll see `Google.com` page not found error
